# TP3: How Much Did It Rain? II

https://www.kaggle.com/c/how-much-did-it-rain-ii


Car cancellation guide:
https://github.com/numb3r33/Kaggle-Competitions/blob/master/cars-cancellation/cars_cancellation.ipynb


En pluviométrie, pour mesurer les hauteurs des précipitations on utilise des jauges qui receuillent la pluie. On est alors capable de déterminer la quantité d'eau tombée durant un intervalle de temps donné, ici une heure.
Cependant, les jauges ne peuvent pas couvrir l'ensemble des lieux que l'on souhaite observer. On utilise alors des radars et on estime la hauteur des précipitations à partir de leurs relevés.
Cependant, ces estimations correspondent mal aux mesures effectués sur les jauges.
L'objectif de ce challenge est de fournir un meilleur estimateur basée sur les relevés des radars.

# Imports

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sb

from sklearn import linear_model
from sklearn import cross_validation
from sklearn.metrics import mean_absolute_error

# Chargement des données

Les données sont des séquences de relevés de capteurs sur une durée d'une heure à des temps variables.
A chaque séquence est associée un identifiant et la distance du capteur à une jauge dont il faut prédire les mesures à la fin de l'heure.
Un élément de la séquence contient les différentes mesures effectués par le radar à un instant donné.

In [3]:
dfTrain = pd.read_csv('./data/train.csv')

# Premier aperçu

On commence par regarder succintement les données. On cherche à répondre à des questions d'ordre général sur les données:
Nombre de séquences et nombre de relevés, nombre de dimension, il y a-t-il des dimensions catégorielles à traiter et y a-t-il des données manquantes?

Un premier résumé permet de dire qu'il y a 13765201 entrées de 24 dimensions (23 si on exclu la quantité a prédire), toutes numériques.
On voit aussi en extrayant la tête et la fin de la table qu'il y a un certain nombre de données manquantes et qu'il y a 1180945 séquences.

In [6]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13765201 entries, 0 to 13765200
Data columns (total 24 columns):
Id                       int64
minutes_past             int64
radardist_km             float64
Ref                      float64
Ref_5x5_10th             float64
Ref_5x5_50th             float64
Ref_5x5_90th             float64
RefComposite             float64
RefComposite_5x5_10th    float64
RefComposite_5x5_50th    float64
RefComposite_5x5_90th    float64
RhoHV                    float64
RhoHV_5x5_10th           float64
RhoHV_5x5_50th           float64
RhoHV_5x5_90th           float64
Zdr                      float64
Zdr_5x5_10th             float64
Zdr_5x5_50th             float64
Zdr_5x5_90th             float64
Kdp                      float64
Kdp_5x5_10th             float64
Kdp_5x5_50th             float64
Kdp_5x5_90th             float64
Expected                 float64
dtypes: float64(22), int64(2)

In [7]:
dfTrain.head(10)

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
1,1,16,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
2,1,25,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
3,1,35,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
4,1,45,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
5,1,55,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
6,2,1,2,9.0,5.0,7.5,10.5,15.0,10.5,16.5,...,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016
7,2,6,2,26.5,22.5,25.5,31.5,26.5,26.5,28.5,...,1.005000,0.0625,-0.1875,0.2500,0.6875,NaN,NaN,NaN,1.409988,1.016
8,2,11,2,21.5,15.5,20.5,25.0,26.5,23.5,25.0,...,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,NaN,-0.350006,1.759994,1.016
9,2,16,2,18.0,14.0,17.5,21.0,20.5,18.0,20.5,...,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016


In [5]:
dfTrain.tail(10)

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
13765191,1180945,14,9,33.5,NaN,32.5,46.5,35.5,6.5,35.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765192,1180945,19,9,43.5,9.5,39.0,49.5,43.5,15.5,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765193,1180945,24,9,31.5,19.5,31.5,41.0,31.5,20.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765194,1180945,28,9,24.0,25.5,31.0,41.0,24.0,25.5,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765195,1180945,33,9,25.5,22.0,27.0,31.0,29.0,25.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765196,1180945,38,9,33.0,19.5,25.5,36.5,33.0,20.5,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765197,1180945,42,9,33.0,21.0,30.5,37.0,36.5,22.0,33.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765198,1180945,47,9,29.5,10.0,26.0,30.5,31.0,16.5,26.0,...,1.051667,1.75,NaN,0.750,3.0000,13.379990,NaN,NaN,13.379990,8.636004
13765199,1180945,52,9,19.0,NaN,15.5,26.5,19.0,NaN,16.5,...,1.051667,NaN,NaN,NaN,2.8125,NaN,NaN,NaN,NaN,8.636004
13765200,1180945,57,9,7.5,NaN,10.0,13.0,14.5,10.0,12.5,...,1.051667,0.00,-1.125,0.375,3.2500,6.069992,NaN,-8.029999,6.069992,8.636004


## Distribution des données

On s'intéresse à la distribution des données, en particulier la quantité de données manquantes et la distribution de la quantité à prédire.

La ligne count nous permet d'évaluer que pour la plupart des colonnes, environ la moitié des données sont manquantes.
La distribution de la colonne Expected est particulière dans le sens où les 3 premiers quartiles sont très bas (respectivement 0.25, 1.02 et 3.81) alors que la moyenne est à 108.63 et le maximum à 33017.73.
C'est-à-dire que la grande majorité des valeurs sont très petites et les grandes valeurs sont extrêmement grandes.
Ceci explique aussi le choix d'une pénalisation sur la norme L1 plutot que L2 qui risquerait de donner un poids démesuré à ces valeurs extrêmes.

In [10]:
dfTrain.describe()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
count,13765201.000000,13765201.000000,13765201.000000,6349375.000000,5283988.000000,6356482.000000,7551281.000000,6716343.000000,5755673.000000,6711663.000000,...,5905584.000000,4934916.000000,4133154.000000,4936568.000000,5905584.000000,4182635.000000,3428782.000000,4187281.000000,5052776.000000,13765201.000000
mean,592336.986614,29.523733,11.067943,22.926658,19.952271,22.610287,25.898461,24.711081,22.158238,24.420753,...,1.015272,0.536709,-0.719008,0.337622,2.072870,0.035452,-3.482325,-0.473655,4.079836,108.626306
std,340856.086254,17.308131,4.206618,10.355157,9.208166,10.053000,11.109579,10.689622,9.702705,10.424526,...,0.048616,1.510399,1.006068,0.938644,1.670194,3.869725,2.792120,2.263046,4.147337,548.605805
min,1.000000,0.000000,0.000000,-31.000000,-32.000000,-32.000000,-28.500000,-32.000000,-31.000000,-27.500000,...,0.208333,-7.875000,-7.875000,-7.875000,-7.875000,-96.040000,-80.790000,-78.770004,-100.200005,0.010000
25%,296897.000000,15.000000,9.000000,16.000000,14.000000,16.000000,18.000000,17.500000,16.000000,17.500000,...,0.998333,-0.187500,-1.125000,-0.062500,1.062500,-1.410004,-4.580002,-0.710007,2.069992,0.254000
50%,592199.000000,30.000000,11.000000,22.500000,20.000000,22.500000,25.500000,24.000000,22.000000,24.000000,...,1.011667,0.375000,-0.625000,0.250000,1.687500,0.000000,-2.820007,0.000000,3.519989,1.016000
75%,889582.000000,44.000000,14.000000,29.500000,26.000000,29.000000,33.500000,31.500000,28.500000,31.500000,...,1.051667,1.062500,-0.187500,0.687500,2.625000,1.750000,-1.760010,0.349991,5.639999,3.810002
max,1180945.000000,59.000000,21.000000,71.000000,62.500000,69.000000,72.500000,92.500000,66.000000,71.000000,...,1.051667,7.937500,7.937500,7.937500,7.937500,179.750000,3.519989,12.800003,144.600000,33017.730000



## Données manquantes

Il y aurait 38% de données manquantes
https://www.kaggle.com/c/how-much-did-it-rain-ii/forums/t/16572/38-missing-data

Cependant, l'administrateur du challenge précise que les séquences pour lesquelles toutes les Ref (colonne \#4) sont manquantes ne seront pas prises en compte.
https://www.kaggle.com/c/how-much-did-it-rain-ii/forums/t/16622/ignored-ids

Dans un premier temps, on se contente de supprimer toutes les lignes contenant des données manquantes. On vérifie qu'on a toujours assez de données pour entrainer des premiers modèles.

In [42]:
dfTrain = dfTrain.dropna()
dfTrain.describe()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
count,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.00000,...,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000,2769088.000000
mean,586194.607283,29.389271,8.463606,26.589885,22.352663,26.558676,31.265658,29.175956,25.587505,29.07478,...,1.006521,0.408317,-0.551204,0.400473,1.603434,0.035481,-3.358400,0.017594,4.018159,12.224754
std,338797.830452,17.173542,3.662127,8.675702,8.118536,8.125674,8.469171,8.726032,8.288342,8.30835,...,0.019291,0.991610,0.732911,0.512091,0.885389,3.067558,2.657847,1.085190,2.786083,156.164185
min,2.000000,0.000000,0.000000,-16.000000,-22.500000,-14.000000,-8.000000,-12.000000,-12.000000,-7.00000,...,0.745000,-7.875000,-7.875000,-4.687500,-1.500000,-57.070000,-66.650000,-16.970001,-4.540008,0.010000
25%,290344.750000,15.000000,6.000000,20.500000,17.000000,21.000000,25.500000,23.000000,20.000000,23.00000,...,0.995000,-0.125000,-0.937500,0.062500,0.937500,-1.410004,-4.400009,-0.350006,2.110001,0.508000
50%,594954.500000,29.000000,9.000000,26.000000,22.000000,26.000000,31.000000,28.500000,25.500000,28.50000,...,0.998333,0.375000,-0.500000,0.312500,1.437500,0.000000,-2.809998,0.000000,3.419998,1.778001
75%,879038.250000,44.000000,11.000000,32.000000,28.000000,32.000000,37.000000,35.000000,31.000000,34.50000,...,1.011667,0.937500,-0.125000,0.687500,2.062500,1.409988,-1.710007,0.699997,5.279999,4.318002
max,1180928.000000,59.000000,15.000000,70.000000,62.500000,65.500000,70.500000,88.500000,66.000000,69.50000,...,1.051667,7.937500,7.937500,7.937500,7.937500,77.399994,3.169998,10.699997,118.570010,33017.730000


In [35]:
ids = dfTrain['Id'].unique()
print "nombre de séquences:", len(ids)

nombre de séquences: 393313


## Premier modèle

On entraîne un premier modèle de régression linéaire, très simple, qui prend en entré un relevé ponctuel.
Comme on dispose de beaucoup de relevés et que l'apprentissage est long, on va sous échantillonner les données.

In [46]:
sample_size = len(dfTrain) / 10

sdfTrain = dfTrain.loc[np.random.permutation(dfTrain.index)[:sample_size]]

y = sdfTrain['Expected']
x = sdfTrain.drop(['Expected', 'Id'],1)

In [47]:
len(y)

276908

In [68]:
regr = linear_model.LinearRegression()

scores = np.zeros(5)

k = 0
for idEx in cross_validation.KFold(len(x.index), n_folds=5):
    xFoldTrain = x.iloc[idEx[0]]
    yFoldTrain = y.iloc[idEx[0]]
    xFoldTest = x.iloc[idEx[1]]
    yFoldTest = y.iloc[idEx[1]]
    regr.fit(xFoldTrain, yFoldTrain)
    scores[k] = mean_absolute_error(regr.predict(xFoldTest), yFoldTest)
    k = k + 1
#scores = cross_validation.cross_val_score(regr, x, y, cv=5, scoring=mean_absolute_error)

In [1]:
print scores.mean()
print scores.std()

NameError: name 'scores' is not defined